### Extract field from invoices using GPT-4 Vision

In [1]:
#Install Dependencies
%pip install python-dotenv
%pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
from dotenv import load_dotenv
import os
import glob
import itertools
from openai import AzureOpenAI
import base64
from mimetypes import guess_type
import json
import pandas as pd

#### Get config details

In [3]:
# Load environment variables
load_dotenv('../config.env')

# Get the connection strings and other secrets
api_base = os.getenv('AOAI_ENDPOINT')
api_key = os.getenv('AOAI_KEY')
deployment_name = os.getenv('AOAI_DEPLOYMENT_NAME')
api_version= os.getenv('AOAI_API_VERSION')
img_path = os.getenv('IMAGE_PATH')

# Print to verify
print(f'Azure OpenAI Url: {api_base}')
print(f'Azure OpenAI Deployment Name: {deployment_name}')
print(f'Azure OpenAI API Version: {api_version}')
print(f'Image path: {img_path}')

Azure OpenAI Url: https://oai-slinvoice-azureai-001.openai.azure.com/
Azure OpenAI Deployment Name: gpt-4-vision
Azure OpenAI API Version: 2023-12-01-preview
Image path: invoice/png/15-05-2024/


#### Helper functions

In [4]:
# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Function to create or update dataframe
def create_or_update_dataframe(data):
    global df
    df1 = pd.json_normalize(data)

    # If df is empty, create a new DataFrame. Otherwise, append to the existing DataFrame
    if df.empty:
        df = df1
    else:
        merged_df = pd.concat([df, df1], ignore_index=True, sort=False)
        df = merged_df
    df.head()
    return df


# Process Invoice
def process_invoice(files, client, prompt):
    # Base messages list
    messages = [
        { "role": "system", "content": prompt },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Please extarct relevant fileds from the invoice"
            }
        ] } 
    ]

    #For each file get the image url and add image_url section to the base message
    for file in files:
        # Get image data url
        image = local_image_to_data_url(f'{img_path}{file}')
        image_url_section = { 
        "type": "image_url",
        "image_url": {
            "url": image
            }
        }
        # Add image url to the message
        messages[1]['content'].append(image_url_section)

    # Pass the invoice data to gpt-4v
    response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=4000,
    temperature=0)


    # Convert the response to a dictionary
    response_dict = response.to_dict()

    # Extract the 'choices' field which contains the model's responses
    choices = response_dict.get('choices')

    # Get the text of the first response
    response_text = choices[0].get('message').get('content') if choices else None
    
    response_text=response_text.replace("```json\n", "").replace("\n```", "")

    # Load the JSON string into a Python dictionary
    invoice_details = json.loads(response_text)

    return invoice_details
   
   




#### Set up OpenAI Client & prompt

In [5]:
# OpenAI client
client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

# Define System Prompt
prompt = ''' 
You are a helpul assistant who understands various invoice formats.
Given an invoice, which could be a single image or multiple images, your task is to extract specific fields from the invoice images. 
The fields to be extracted are as follows:

    Vendor Name
    Vendor VAT Registration Number
    Invoice or Credit
    Invoice Number
    Credit Reference
    Invoice Date
    Currency
    Net Amount
    Tax Amount
    Freight
    Misc Charges
    Total Amount
    Purchase Order Number

###Guidelines:###
	If an invoice contains multiple Purchase Orders (POs), the Purchase Order field will repeat. This field **MUST BE** kept as an array.

	The Vendor Name and Vendor VAT Registration Number are associated with the entity issuing the invoice, not with the 'ship to', 'sold to', 'customer', or 'billed to' fields. The Vendor Name is not 'RENISHAW', 'RENISHAW PLC' or anything similar.
	You **MUST** return the Vendor VAT Registration Number as specified in the invoice image.

	The Invoice Date **MUST** alwyas be returned in the 'dd/MM/yyyy' format.
	The Currency field **MUST** always be returned as an ISO 4217 code.

	You **MUST** refrain from fabricating any text or numerical value that is not present in the image file.
	You **MUST NOT** calculate the Net Amount, Tax Amount, Freight, Misc Charges, and Total Amount. These values **MUST BE** based on the text from the image. The Total Amount is always the total amount due.
    Please be aware that Freight might be present as a line item in some invoices. You ** MUST NOT** add Freight as Misc Charges.

	If a field is not present, You **MUST** return the field name without any value.

	The result should be returned in the following JSON format and **MUST NOT** include any other text:
		{
		"VendorName": "",
		"VendorVATRegistrationNumber": "",
		"InvoiceOrCredit": "",
		"InvoiceNumber": "",
		"CreditReference": "",
		"InvoiceDate": "",
		"Currency": "",
		"NetAmount": "",
		"TaxAmount": "",
		"Freight": "",
		"MiscCharges": "",
		"TotalAmount": "",
		"Orders": [
			{
				"PONumber": ""
			}
		]
	}
###
'''

#### Process files in group & invoke openAI

In [6]:
# Global variable for the DataFrame
df = pd.DataFrame()

# Get all png files in the img_path directory
files = glob.glob(f'{img_path}*.png')

# Remove img_path from all file names
files = [file.replace(img_path, '') for file in files]

# Group files based on the name part (after '_')
groups = itertools.groupby(sorted(files, key=lambda x: x.split('_')[1]), key=lambda x: x.split('_')[1])

for name, group in groups:
    # Sort files in each group based on the iteration part (before '_')
    sorted_group = sorted(list(group), key=lambda x: int(x.split('_')[0]))
    # Call your processing function for each group of files
    response=process_invoice(sorted_group, client, prompt)
    # Add file to the dictionry
    response['FileUrl'] = name
    print(response)
    print("\n")
    df = create_or_update_dataframe(response)




{'VendorName': 'Shenzhen Indus-connector Limited', 'VendorVATRegistrationNumber': 'CN 138025302727', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': 'IC210130-01', 'CreditReference': '', 'InvoiceDate': '25/02/2021', 'Currency': 'USD', 'NetAmount': '2,850.00', 'TaxAmount': '', 'Freight': '0.00', 'MiscCharges': '', 'TotalAmount': '2,850.00', 'Orders': [{'PONumber': ''}], 'FileUrl': '681409.png'}


{'VendorName': 'Farnell', 'VendorVATRegistrationNumber': 'GB 169 6803 22', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': '5792718', 'CreditReference': '', 'InvoiceDate': '20/10/2021', 'Currency': 'GBP', 'NetAmount': '14856.97', 'TaxAmount': '2971.39', 'Freight': '', 'MiscCharges': '', 'TotalAmount': '17828.36', 'Orders': [{'PONumber': 'P003579185'}], 'FileUrl': '714365.png'}


{'VendorName': 'SWEETNAM & BRADLEY LTD', 'VendorVATRegistrationNumber': '930032965', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': '74840', 'CreditReference': '', 'InvoiceDate': '17/12/2021', 'Currency': 'GBP', 'NetAmoun

In [7]:
df.head()

,VendorName,VendorVATRegistrationNumber,InvoiceOrCredit,InvoiceNumber,CreditReference,InvoiceDate,Currency,NetAmount,TaxAmount,Freight,MiscCharges,TotalAmount,Orders,FileUrl
0,Shenzhen Indus-connector Limited,CN 138025302727,Invoice,IC210130-01,,25/02/2021,USD,"2,850.00",,0.00,,"2,850.00",[{'PONumber': ''}],681409.png
1,Farnell,GB 169 6803 22,Invoice,5792718,,20/10/2021,GBP,14856.97,2971.39,,,17828.36,[{'PONumber': 'P003579185'}],714365.png
2,SWEETNAM & BRADLEY LTD,930032965,Invoice,74840,,17/12/2021,GBP,1575.04,315.01,0.00,,1890.05,[{'PONumber': 'PU03581000'}],724393.png
3,TR Fastenings Limited,GB363096465,Invoice,621185,,17/12/2021,GBP,879.34,175.87,,,"1,055.21",[{'PONumber': 'PU03585370'}],724454.png
4,ERIKS Industrial Services Ltd.,GB 277 2632 40,Invoice,01808896,,23/12/2021,GBP,2980.00,596.00,,,3576.00,[{'PONumber': 'PU03576899'}],726035.png


In [8]:
from datetime import datetime
#Save results
# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

# Define the directory path
dir_path = os.path.join('analyzed','no-ocr', timestamp)

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)

# Define the file path
file_path = os.path.join(dir_path, 'result.csv')
# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)